# Homework 3

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.columns = df.columns.str.lower().str.replace(' ','_')
df.head().T

,0,1,2,3,4
customerid,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
seniorcitizen,0,0,0,0,0
partner,Yes,No,No,No,No
dependents,No,No,No,No,No
tenure,1,34,2,45,2
phoneservice,No,Yes,Yes,No,Yes
multiplelines,No phone service,No,No,No phone service,No
internetservice,DSL,DSL,DSL,DSL,Fiber optic
onlinesecurity,No,Yes,Yes,Yes,No


In [6]:
categorical_columns = list(df.dtypes[df.dtypes=='object'].index)

In [7]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [8]:
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')

In [9]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')

In [10]:
df.churn = (df.churn == 'yes').astype(int)

In [11]:
df.totalcharges = df.totalcharges.fillna(0)

## prepare data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
full_train ,test_set = train_test_split(df, test_size =0.2, random_state= 1)
train_set ,val_set = train_test_split(full_train, test_size =0.25, random_state= 1)

In [14]:
train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [15]:
y_train = train_set['churn']
y_val = val_set['churn']
y_test = test_set['churn']

In [16]:
del train_set['churn']
del val_set['churn']
del test_set['churn']

## EDA

In [17]:
from IPython.display import display

In [18]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]
numerical = ['tenure', 'monthlycharges', 'totalcharges']


In [19]:
df.churn.value_counts(normalize = True)

churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

In [20]:
full_train
global_churn = full_train.churn.mean()
global_churn

np.float64(0.26996805111821087)

In [21]:
for c in categorical:
    df_group = full_train.groupby(c).churn.agg(['count','mean'])
    df_group['risk'] = df_group['mean']/ global_churn
    display(df_group)
    print()

,count,mean,risk
gender,,,
female,2796,0.276824,1.025396
male,2838,0.263214,0.974980


,count,mean,risk
seniorcitizen,,,
0,4722,0.242270,0.897403
1,912,0.413377,1.531208


,count,mean,risk
partner,,,
no,2932,0.329809,1.221659
yes,2702,0.205033,0.759472


,count,mean,risk
dependents,,,
no,3968,0.313760,1.162212
yes,1666,0.165666,0.613651


,count,mean,risk
phoneservice,,,
no,547,0.241316,0.893870
yes,5087,0.273049,1.011412


...

,count,mean,risk
streamingtv,,,
no,2246,0.342832,1.269897
no_internet_service,1221,0.077805,0.288201
yes,2167,0.302723,1.121328


,count,mean,risk
streamingmovies,,,
no,2213,0.338906,1.255358
no_internet_service,1221,0.077805,0.288201
yes,2200,0.307273,1.138182


,count,mean,risk
contract,,,
month-to-month,3104,0.431701,1.599082
one_year,1186,0.120573,0.446621
two_year,1344,0.028274,0.104730


,count,mean,risk
paperlessbilling,,,
no,2313,0.172071,0.637375
yes,3321,0.338151,1.252560


,count,mean,risk
paymentmethod,,,
bank_transfer_(automatic),1219,0.168171,0.622928
credit_card_(automatic),1217,0.164339,0.608733
electronic_check,1893,0.455890,1.688682
mailed_check,1305,0.193870,0.718121


## Mutual Information

In [22]:
from sklearn.metrics import mutual_info_score

In [23]:
mutual_info_score(full_train['gender'],full_train['churn'])

0.0001174846211139946

In [24]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, full_train.churn)

In [25]:
mi = full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

## Correlation

In [26]:
full_train[numerical].corrwith(full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

## one-Hot encoder

In [71]:
from sklearn.feature_extraction import DictVectorizer

In [72]:
full_train = full_train.reset_index(drop=True)

In [73]:
train_dicts = train_set[categorical+ numerical].to_dict(orient='records')

In [74]:
dv = DictVectorizer(sparse= False)

In [75]:
dv.fit(dicts)

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


In [76]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

In [84]:
X_train = dv.transform(train_dicts)

In [85]:
val_dicts = val_set[categorical+numerical].to_dict(orient='records')

In [86]:
X_val = dv.transform(val_dicts)

## Logistic Regression

In [87]:
from sklearn.linear_model import LogisticRegression

In [88]:
model = LogisticRegression()

In [89]:
model.fit(X_train,y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [93]:
model.coef_.round(3)
model.intercept_

array([-0.10913177])

In [94]:
model.predict(X_train)

array([0, 1, 1, ..., 1, 0, 1], shape=(4225,))

In [95]:
model.predict_proba(X_train)

array([[0.90416494, 0.09583506],
       [0.3210407 , 0.6789593 ],
       [0.36668394, 0.63331606],
       ...,
       [0.46853433, 0.53146567],
       [0.9576116 , 0.0423884 ],
       [0.30173516, 0.69826484]], shape=(4225, 2))

In [98]:
y_pred = model.predict_proba(X_val)[:,1]

In [108]:
churn_decision = (y_pred > 0.5)

In [109]:
val_set[churn_decision]

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
3,8433-wxgna,male,0,no,no,2,yes,no,fiber_optic,yes,no,no,no,no,no,month-to-month,yes,electronic_check,75.70,189.20
8,3440-jpscl,female,0,no,no,6,yes,no,fiber_optic,no,no,yes,yes,yes,yes,month-to-month,yes,mailed_check,99.95,547.65
11,2637-fkfsy,female,0,yes,no,3,yes,no,dsl,no,no,no,no,no,no,month-to-month,yes,mailed_check,46.10,130.15
12,7228-omtpn,male,0,no,no,4,yes,no,fiber_optic,no,no,no,no,yes,yes,month-to-month,yes,electronic_check,88.45,370.65
19,6711-fldfb,female,0,no,no,7,yes,yes,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,74.90,541.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,5976-jcjrh,male,0,yes,no,10,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.30,738.20
1398,2034-cgrhz,male,1,no,no,24,yes,yes,fiber_optic,no,yes,yes,no,yes,yes,month-to-month,yes,credit_card_(automatic),102.95,2496.70
1399,5276-kqwhg,female,1,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,69.60,131.65
1407,6521-yytyi,male,0,no,yes,1,yes,yes,fiber_optic,no,no,no,no,yes,yes,month-to-month,yes,electronic_check,93.30,93.30
